In [1]:
%%capture
!pip -q install git+https://github.com/huggingface/transformers
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate
!pip -q install langchain
!pip install faiss-gpu
!pip install einops
!pip install --upgrade --quiet  langchain-community chromadb bs4 qdrant-client
!pip install langchainhub
!pip install sentence-transformers
!pip install tqdm
!pip install rouge_score nltk bert_score

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/DATASCI266/project')

In [6]:
from summary_utils import SummaryEvaluator, DatasetManager, SummaryModel

In [7]:
import os
import re
import torch
from datasets import load_dataset
from tqdm import tqdm
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain.docstore.in_memory import InMemoryDocstore
from langchain import PromptTemplate, LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer , AutoModelForCausalLM
from transformers import pipeline, BitsAndBytesConfig

In [8]:
summary_evaluator = SummaryEvaluator()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
# Load the XSum dataset
data_manager = DatasetManager(sample_size=200)
dataset = data_manager.load_sampled_dataset(dataset_label="test")

# Initialize the Embeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

The repository for xsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/xsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
%%capture

quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         llm_int4_enable_fp32_cpu_offload=True)


llm_mistral_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    torch_dtype=torch.float32,
    device_map='auto',
    quantization_config=quantization_config
)

llm_mistral_tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

Unused kwargs: ['llm_int4_enable_fp32_cpu_offload']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


In [11]:
mistral_pipe = pipeline(
    "text-generation",
    model=llm_mistral_model,
    tokenizer=llm_mistral_tokenizer,
    max_length=1000,
    temperature=0.6,
    top_p=0.95,
    do_sample=True,
    repetition_penalty=1.2
)
mistral_pipe.model.config.pad_token_id = mistral_pipe.model.config.eos_token_id


In [12]:
mistral_llm_lc = HuggingFacePipeline(pipeline=mistral_pipe)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [13]:
class RAG_Summarization_Model:
  def __init__(self, chunk_size=200, chunk_overlap=50):
    self.vector_store = None
    self.rag_chain = None
    self.chunk_size = chunk_size
    self.chunk_overlap = chunk_overlap

  def fit(self, dataset):
    # Function to split documents and include metadata
    def create_documents_with_metadata(dataset, chunk_size=200, chunk_overlap=50):
      # Initialize the text splitter
      text_splitter = RecursiveCharacterTextSplitter(
          chunk_size=chunk_size,  # Adjust based on your needs
          chunk_overlap=chunk_overlap  # Adjust based on your needs
      )
      documents = []
      for example in dataset:
          text = example['document']
          # Split the text into chunks while including metadata
          chunks = text_splitter.split_text(text)
          for chunk in chunks:
              documents.append(Document(page_content=chunk, metadata={"id": example['id']}))
      return documents
    # Create documents with metadata
    documents = create_documents_with_metadata(dataset, self.chunk_size, self.chunk_overlap)
    # Create a FAISS vector store
    self.vector_store = FAISS.from_documents(documents, embeddings)

    rag_template = """[INST] Give me a short one-sentence summarization with the following context: {context}[/INST]"""

    rag_prompt = ChatPromptTemplate.from_template(rag_template)

    self.rag_chain = (
        {"context": lambda inputs: self.format_docs(self.query_vector_store(inputs['query'], inputs['id'], k=inputs['k'], fetch_k=inputs['fetch_k'])),
        "question": RunnablePassthrough()}
        | rag_prompt
        | mistral_llm_lc
    )

  def query_vector_store(self, query_text, summary_id, k=3, fetch_k=20):
    # Embed the query text
    query_vector = embeddings.embed_query(query_text)
    # Search the vector store
    docs = self.vector_store.similarity_search_by_vector(query_vector, filter=dict(id=summary_id), k=k, fetch_k=fetch_k)
    return docs

  def format_docs(self, docs):
      return "\n\n".join(doc.page_content for doc in docs)

  def predict(self, dataset, k, fetch_k):
    results = []
    i = 1
    for example in tqdm(dataset):
        print(i)
        text = example['document']
        sentences = re.split(r'[.?!]\s+', text)
        example_query = sentences[0]
        summary_w_prompt = self.rag_chain.invoke({"query": example_query, "id": example['id'], "k":k, 'fetch_k':fetch_k})
        summary = summary_w_prompt.split("[/INST] ",1)[1]
        results.append(summary)
        i += 1
    return results

In [14]:
references = [example['summary'] for example in dataset]

In [15]:
rag_model = RAG_Summarization_Model(chunk_size=200, chunk_overlap=50)
rag_model.fit(dataset)
predictions1 = rag_model.predict(dataset, k=3, fetch_k=50)
predictions2 = rag_model.predict(dataset, k=5, fetch_k=50)
predictions3 = rag_model.predict(dataset, k=10, fetch_k=50)
predictions4 = rag_model.predict(dataset, k=15, fetch_k=50)

  0%|          | 0/200 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1


  0%|          | 1/200 [00:06<22:10,  6.69s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


2


  1%|          | 2/200 [00:10<16:18,  4.94s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

 99%|█████████▉| 198/200 [19:42<00:13,  6.51s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


199


100%|█████████▉| 199/200 [19:46<00:05,  5.67s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


200


100%|██████████| 200/200 [19:50<00:00,  5.95s/it]


In [16]:
predictions1[0]

'In April 2013, minibus driver James Johnson was sentenced to more than six years in jail for causing the death of his friend Bethany Jones, 18, and injuring Sarah Johnson and several others in a crash on the M62.'

In [17]:
print("chunk_size=200, chunk_overlap=50")
print("k=3, fetch_k=50")
summary_evaluator.evaluate(references, predictions1)
print("k=5, fetch_k=50")
summary_evaluator.evaluate(references, predictions2)
print("k=10, fetch_k=50")
summary_evaluator.evaluate(references, predictions3)
print("k=15, fetch_k=50")
summary_evaluator.evaluate(references, predictions4)

chunk_size=200, chunk_overlap=50
k=3, fetch_k=50
Average ROUGE score: {'rouge1': 0.20067657591984825, 'rouge2': 0.030829305430531403, 'rougeL': 0.1393695206692511}
Average BLEU score: 0.018635631447774647


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.5277141332626343
Average VECTOR_SIMILARITY score: 0.5242981314659119
k=15, fetch_k=50
Average ROUGE score: {'rouge1': 0.21041360384940463, 'rouge2': 0.03841223178030937, 'rougeL': 0.14992642915651136}
Average BLEU score: 0.02170808862742337


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.5257427096366882
Average VECTOR_SIMILARITY score: 0.5236860513687134


{'rouge': {'rouge1': 0.21041360384940463,
  'rouge2': 0.03841223178030937,
  'rougeL': 0.14992642915651136},
 'bleu': 0.02170808862742337,
 'bertscore': 0.5257427096366882,
 'vector_similarity': 0.5236860513687134}

In [18]:
rag_model = RAG_Summarization_Model(chunk_size=100, chunk_overlap=50)
rag_model.fit(dataset)
predictions1 = rag_model.predict(dataset, k=3, fetch_k=50)
predictions2 = rag_model.predict(dataset, k=5, fetch_k=50)
predictions3 = rag_model.predict(dataset, k=10, fetch_k=50)
predictions4 = rag_model.predict(dataset, k=15, fetch_k=50)

  0%|          | 0/200 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1


  0%|          | 1/200 [00:03<13:04,  3.94s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


2


  1%|          | 2/200 [00:06<11:06,  3.37s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

 99%|█████████▉| 198/200 [18:04<00:09,  4.90s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


199


100%|█████████▉| 199/200 [18:08<00:04,  4.68s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


200


100%|██████████| 200/200 [18:12<00:00,  5.46s/it]


In [19]:
print("chunk_size=100, chunk_overlap=50")
print("k=3, fetch_k=50")
summary_evaluator.evaluate(references, predictions1)
print("k=5, fetch_k=50")
summary_evaluator.evaluate(references, predictions2)
print("k=10, fetch_k=50")
summary_evaluator.evaluate(references, predictions3)
print("k=15, fetch_k=50")
summary_evaluator.evaluate(references, predictions4)

chunk_size=100, chunk_overlap=50
k=3, fetch_k=50
Average ROUGE score: {'rouge1': 0.18403856483837633, 'rouge2': 0.027850526677619572, 'rougeL': 0.13827544107972295}
Average BLEU score: 0.018962427171773138


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.5032695531845093
Average VECTOR_SIMILARITY score: 0.4393357038497925
k=5, fetch_k=50
Average ROUGE score: {'rouge1': 0.19552128952260744, 'rouge2': 0.03156783276465688, 'rougeL': 0.13876156248472507}
Average BLEU score: 0.018979791326327954


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.511831521987915
Average VECTOR_SIMILARITY score: 0.4792373478412628
k=10, fetch_k=50
Average ROUGE score: {'rouge1': 0.20205693826710472, 'rouge2': 0.03451986353627953, 'rougeL': 0.14282762296267587}
Average BLEU score: 0.020734946649904856


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.5194266438484192
Average VECTOR_SIMILARITY score: 0.4959244430065155
k=15, fetch_k=50
Average ROUGE score: {'rouge1': 0.20833097253797933, 'rouge2': 0.03919043696451096, 'rougeL': 0.1532386407328089}
Average BLEU score: 0.021446503719401277


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.5228315591812134
Average VECTOR_SIMILARITY score: 0.5066124796867371


{'rouge': {'rouge1': 0.20833097253797933,
  'rouge2': 0.03919043696451096,
  'rougeL': 0.1532386407328089},
 'bleu': 0.021446503719401277,
 'bertscore': 0.5228315591812134,
 'vector_similarity': 0.5066124796867371}

In [20]:
rag_model = RAG_Summarization_Model(chunk_size=200, chunk_overlap=0)
rag_model.fit(dataset)
predictions1 = rag_model.predict(dataset, k=3, fetch_k=50)
predictions2 = rag_model.predict(dataset, k=5, fetch_k=50)
predictions3 = rag_model.predict(dataset, k=10, fetch_k=50)
predictions4 = rag_model.predict(dataset, k=15, fetch_k=50)

  0%|          | 0/200 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1


  0%|          | 1/200 [00:05<19:23,  5.85s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


2


  1%|          | 2/200 [00:08<13:43,  4.16s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

 99%|█████████▉| 198/200 [20:20<00:13,  6.55s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


199


100%|█████████▉| 199/200 [20:24<00:05,  5.69s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


200


100%|██████████| 200/200 [20:28<00:00,  6.14s/it]


In [21]:
print("chunk_size=200, chunk_overlap=0")
print("k=3, fetch_k=50")
summary_evaluator.evaluate(references, predictions1)
print("k=5, fetch_k=50")
summary_evaluator.evaluate(references, predictions2)
print("k=10, fetch_k=50")
summary_evaluator.evaluate(references, predictions3)
print("k=15, fetch_k=50")
summary_evaluator.evaluate(references, predictions4)

chunk_size=200, chunk_overlap=0
k=3, fetch_k=50
Average ROUGE score: {'rouge1': 0.2062778132192565, 'rouge2': 0.03175558377070006, 'rougeL': 0.1464080303024817}
Average BLEU score: 0.0204950944238727


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.5200604200363159
Average VECTOR_SIMILARITY score: 0.49854788184165955
k=5, fetch_k=50
Average ROUGE score: {'rouge1': 0.21056844682598913, 'rouge2': 0.03668999045961707, 'rougeL': 0.14748790436037665}
Average BLEU score: 0.019743831849401948


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.522504448890686
Average VECTOR_SIMILARITY score: 0.5193471908569336
k=10, fetch_k=50
Average ROUGE score: {'rouge1': 0.2097190443872799, 'rouge2': 0.038017624237110505, 'rougeL': 0.14481114100730924}
Average BLEU score: 0.02083693720604405


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.5252408385276794
Average VECTOR_SIMILARITY score: 0.5220113396644592
k=15, fetch_k=50
Average ROUGE score: {'rouge1': 0.21954353891303718, 'rouge2': 0.043345357971853085, 'rougeL': 0.15397438817461118}
Average BLEU score: 0.020969821981885718


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.5330560207366943
Average VECTOR_SIMILARITY score: 0.5370754599571228


{'rouge': {'rouge1': 0.21954353891303718,
  'rouge2': 0.043345357971853085,
  'rougeL': 0.15397438817461118},
 'bleu': 0.020969821981885718,
 'bertscore': 0.5330560207366943,
 'vector_similarity': 0.5370754599571228}

In [22]:
rag_model = RAG_Summarization_Model(chunk_size=100, chunk_overlap=0)
rag_model.fit(dataset)
predictions1 = rag_model.predict(dataset, k=3, fetch_k=50)
predictions2 = rag_model.predict(dataset, k=5, fetch_k=50)
predictions3 = rag_model.predict(dataset, k=10, fetch_k=50)
predictions4 = rag_model.predict(dataset, k=15, fetch_k=50)

  0%|          | 0/200 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1


  0%|          | 1/200 [00:03<12:40,  3.82s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


2


  1%|          | 2/200 [00:08<13:53,  4.21s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

 99%|█████████▉| 198/200 [17:55<00:10,  5.04s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


199


100%|█████████▉| 199/200 [17:58<00:04,  4.56s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


200


100%|██████████| 200/200 [18:03<00:00,  5.42s/it]


In [23]:
print("chunk_size=100, chunk_overlap=0")
print("k=3, fetch_k=50")
summary_evaluator.evaluate(references, predictions1)
print("k=5, fetch_k=50")
summary_evaluator.evaluate(references, predictions2)
print("k=10, fetch_k=50")
summary_evaluator.evaluate(references, predictions3)
print("k=15, fetch_k=50")
summary_evaluator.evaluate(references, predictions4)

chunk_size=100, chunk_overlap=0
k=3, fetch_k=50
Average ROUGE score: {'rouge1': 0.1967605952948349, 'rouge2': 0.029299779161866445, 'rougeL': 0.14084893734658252}
Average BLEU score: 0.019647185682673


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.504014253616333
Average VECTOR_SIMILARITY score: 0.4522652328014374
k=5, fetch_k=50
Average ROUGE score: {'rouge1': 0.2007900543552514, 'rouge2': 0.03241843924749538, 'rougeL': 0.14156170145337327}
Average BLEU score: 0.01897057632104693


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.5146355032920837
Average VECTOR_SIMILARITY score: 0.4777347445487976
k=10, fetch_k=50
Average ROUGE score: {'rouge1': 0.20810832545962285, 'rouge2': 0.03623784213089343, 'rougeL': 0.1477481995927908}
Average BLEU score: 0.020676565642568444


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.5217396020889282
Average VECTOR_SIMILARITY score: 0.49164122343063354
k=15, fetch_k=50
Average ROUGE score: {'rouge1': 0.2050761857754115, 'rouge2': 0.03661470405603989, 'rougeL': 0.1439059962719931}
Average BLEU score: 0.021530292331361802


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.5212727785110474
Average VECTOR_SIMILARITY score: 0.5040372610092163


{'rouge': {'rouge1': 0.2050761857754115,
  'rouge2': 0.03661470405603989,
  'rougeL': 0.1439059962719931},
 'bleu': 0.021530292331361802,
 'bertscore': 0.5212727785110474,
 'vector_similarity': 0.5040372610092163}

In [24]:
rag_model = RAG_Summarization_Model(chunk_size=150, chunk_overlap=0)
rag_model.fit(dataset)
predictions1 = rag_model.predict(dataset, k=3, fetch_k=50)
predictions2 = rag_model.predict(dataset, k=5, fetch_k=50)
predictions3 = rag_model.predict(dataset, k=10, fetch_k=50)
predictions4 = rag_model.predict(dataset, k=15, fetch_k=50)

  0%|          | 0/200 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1


  0%|          | 1/200 [00:04<13:51,  4.18s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


2


  1%|          | 2/200 [00:07<11:17,  3.42s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

 99%|█████████▉| 198/200 [18:53<00:10,  5.50s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


199


100%|█████████▉| 199/200 [18:59<00:05,  5.40s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


200


100%|██████████| 200/200 [19:03<00:00,  5.72s/it]


In [25]:
print("chunk_size=150, chunk_overlap=0")
print("k=3, fetch_k=50")
summary_evaluator.evaluate(references, predictions1)
print("k=5, fetch_k=50")
summary_evaluator.evaluate(references, predictions2)
print("k=10, fetch_k=50")
summary_evaluator.evaluate(references, predictions3)
print("k=15, fetch_k=50")
summary_evaluator.evaluate(references, predictions4)

chunk_size=150, chunk_overlap=0
k=3, fetch_k=50
Average ROUGE score: {'rouge1': 0.19497802541161832, 'rouge2': 0.03365327927698086, 'rougeL': 0.1406521916438732}
Average BLEU score: 0.020630052016087707


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.5101431012153625
Average VECTOR_SIMILARITY score: 0.46646103262901306
k=5, fetch_k=50
Average ROUGE score: {'rouge1': 0.20055958854662426, 'rouge2': 0.03823467548383068, 'rougeL': 0.14509969405598577}
Average BLEU score: 0.021430934645224102


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.5174753665924072
Average VECTOR_SIMILARITY score: 0.48854610323905945
k=10, fetch_k=50
Average ROUGE score: {'rouge1': 0.21104986047225358, 'rouge2': 0.04061821564582855, 'rougeL': 0.14883151284246574}
Average BLEU score: 0.021392771865600436


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.5244314074516296
Average VECTOR_SIMILARITY score: 0.5235612392425537
k=15, fetch_k=50
Average ROUGE score: {'rouge1': 0.206231014075486, 'rouge2': 0.038902718421680735, 'rougeL': 0.14890892786387638}
Average BLEU score: 0.020700725549185978


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.5225045680999756
Average VECTOR_SIMILARITY score: 0.5131612420082092


{'rouge': {'rouge1': 0.206231014075486,
  'rouge2': 0.038902718421680735,
  'rougeL': 0.14890892786387638},
 'bleu': 0.020700725549185978,
 'bertscore': 0.5225045680999756,
 'vector_similarity': 0.5131612420082092}